In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import spacy
from spacy.lang.en import English

In [15]:
data_train = pd.read_csv("../../data/data_train.csv").drop(["Unnamed: 0"], axis=1)
data_test = pd.read_csv("../../data/data_test.csv").drop(["Unnamed: 0"], axis=1)

nlp = spacy.load('en_core_web_sm')

In [21]:
# Funktion zur Tokenisierung, Lemmatisierung, Stop-Word-Removal der Texte
def preprocess_text(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    num_sentences = len(sentences)
    
    words = []
    lemmas = []
    stops = []

    for token in doc:
        if not token.is_stop and not token.is_punct:
            words.append(token.text)
            lemmas.append(token.lemma_)
        elif not token.is_punct:
            stops.append(token)
    num_words = len(words)
    num_stops = len(stops)

    return sentences, num_sentences, words, num_words, lemmas, stops, num_stops

data_train['sentences'], data_train['num_sentences'], data_train['words'], data_train['num_words'], data_train['lemmas'], data_train['stops'], data_train['num_stops'] = zip(*data_train['text'].apply(preprocess_text))
data_test['sentences'], data_test['num_sentences'], data_test['words'], data_test['num_words'], data_test['lemmas'], data_test['stops'], data_test['num_stops'] = zip(*data_test['text'].apply(preprocess_text))

data_train.to_csv("../../data/data_train_with_features.csv")
data_test.to_csv("../../data/data_test_with_features.csv")

# etwa 4 Minuten

In [22]:
data_train

,text,classification,sentences,num_sentences,words,num_words,lemmas,stops,num_stops
0,Stereo matching is one of the widely used tech...,paper,[Stereo matching is one of the widely used tec...,6,"[Stereo, matching, widely, techniques, inferri...",82,"[Stereo, matching, widely, technique, infer, d...","[is, one, of, the, used, for, from, to, its, a...",62
1,The recent advancements in artificial intellig...,paper,[The recent advancements in artificial intelli...,7,"[recent, advancements, artificial, intelligenc...",154,"[recent, advancement, artificial, intelligence...","[The, in, with, the, amount, of, by, 's, has, ...",76
2,"In this paper, we proposed a novel mutual cons...",paper,"[In this paper, we proposed a novel mutual con...",11,"[paper, proposed, novel, mutual, consistency, ...",178,"[paper, propose, novel, mutual, consistency, n...","[In, this, we, a, to, the, for, The, is, by, t...",89
3,Consistency training has proven to be an advan...,paper,[Consistency training has proven to be an adva...,10,"[Consistency, training, proven, advanced, semi...",143,"[consistency, training, prove, advanced, semi,...","[has, to, be, an, and, in, through, an, of, th...",81
4,"To ensure safety in automated driving, the cor...",paper,"[To ensure safety in automated driving, the co...",13,"[ensure, safety, automated, driving, correct, ...",182,"[ensure, safety, automate, driving, correct, p...","[To, in, the, of, the, the, is, as, as, its, T...",121
...,...,...,...,...,...,...,...,...,...
1995,"The room was large, but with a low ceiling, an...",story,"[The room was large, but with a low ceiling, a...",231,"[room, large, low, ceiling, end, lengthy, broa...",1949,"[room, large, low, ceiling, end, lengthy, broa...","[The, was, but, with, a, and, at, one, of, the...",2634
1996,"""There is nothing more certain than that some ...",story,"[""There is nothing more certain than that some...",104,"[certain, day, mil, record, terrible, disaster...",837,"[certain, day, mil, record, terrible, disaster...","[There, is, nothing, more, than, that, some, w...",1288
1997,In the saloon of the fine Transatlantic liner ...,story,[In the saloon of the fine Transatlantic liner...,129,"[saloon, fine, Transatlantic, liner, Climatus,...",865,"[saloon, fine, transatlantic, liner, Climatus,...","[In, the, of, the, the, two, from, the, at, on...",1168
1998,The trouble about this story is that it really...,story,[The trouble about this story is that it reall...,233,"[trouble, story, ending, Taking, ocean, voyage...",1725,"[trouble, story, ending, take, ocean, voyage, ...","[The, about, this, is, that, it, really, has, ...",2643
